## **Import the TensorFlow library**

This code contains a small script that is used for generating a folder full of images. The idea is to show these images to the OpenMV camera when the training with the OL methos is required. Since the idea is to apply a supervised training is required to have a ground truth forthe computation of the error that the model is doing. This script fills a directory with images of digits and creates a txt file in which all the labels are saved. 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dropout, Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from PIL import Image

import numpy as np
import pandas as pd
import seaborn as sns
import random 
import csv 
import sys
import os
import re
from random import seed

import time
#import os, os.path
import cv2
import glob
from keras import applications
#from keras.applications import vgg19
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score

from numpy.ma.core import size

import torchvision.models as models
from torchvision import transforms
ROOT_PATH = os.path.abspath('')
sys.path.insert(0, ROOT_PATH + '/lib')

# VS Code

# IMPORT AND LOAD MODELS

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from sklearn.cluster import KMeans


import random 
from random import seed

import time


import numpy as np
import matplotlib.pyplot as plt

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')

# Test import
# from lib import simulation_lib
# from lib.simulation_lib import

from lib import Kmeans_lib
from lib.Kmeans_lib import *
from lib.EvalMetrics import *


# LOAD DATASETS

In [27]:
(data_train, label_train),(data_test, label_test) = mnist.load_data() # Load data
print('The original dataset shapes from MNIST are')
print(f'    Train dataset shape: {data_train.shape}')
print(f'    Test dataset shape:  {data_test.shape}')

The original dataset shapes from MNIST are
    Train dataset shape: (60000, 28, 28)
    Test dataset shape:  (10000, 28, 28)


In [28]:
n_samples = 200 # Gets doubled
digits_train = np.zeros((n_samples,28,28))
digits_test = np.zeros((n_samples,28,28))
label_digits_train = np.zeros(n_samples)
label_digits_test = np.zeros(n_samples)

for i in range(0, n_samples):
  n = random.randint(0,len(data_train)-1)
  digits_train[i,:,:] = data_train[n,:,:]
  label_digits_train[i] = label_train[n]
  m = random.randint(0,len(data_test)-1)
  digits_test[i,:,:] = np.copy(data_test[m,:,:])
  label_digits_test[i] = label_test[m]

img_rows, img_cols = 28, 28
digits_train  = digits_train.reshape(digits_train.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
digits_test = digits_test.reshape(digits_test.shape[0], img_rows, img_cols, 1).astype(np.float32) / 255.0
input_shape = (img_rows, img_cols, 1)

print(f'The dataset on which the model will be TRAINED has shape {digits_train.shape}') # da cambiare con il nuovo set
print(f'The dataset on which the model will be TESTED has shape  {digits_test.shape}') # da cambiare con il nuovo set

The dataset on which the model will be TRAINED has shape (200, 28, 28, 1)
The dataset on which the model will be TESTED has shape  (200, 28, 28, 1)


# TRAIN THE MODEL ON THE DIGITS 6-9 (OL METHOD)

In [29]:
'''Function to map the cluster index with the pseudo labels. The function must be run only on the saved_dataset'''
def cluster_to_label2(clusters_features, labels_features, cluster_list, labels_init_list):

  # 1: Compute Confusion matrix (for the saved features)
  cmtx = confusion_matrix2(clusters_features, labels_features, cluster_list, labels_init_list)

  # 2: Find max in each row -> cluster corresponding to each label
  map_idx = np.argmax(cmtx, axis = 1)  

  # Fill dictionary with map
  map_clu2lbl = {}
  map_lbl2clu = {}
  labels_init_list.sort()
  for i in range(0, len(map_idx)):
    map_clu2lbl[map_idx[i]] = labels_init_list[i]
    map_lbl2clu[labels_init_list[i]] = map_idx[i]
  
  # Mapping dictionary
  # map_clu2lbl -> cluster: label
  # map_lbl2clu -> label: cluster

  return map_clu2lbl, map_lbl2clu


'''Function to map the cluster index with the pseudo labels. The function must be run only on the saved_dataset'''
def cluster_to_label(clusters_features, labels_features, cluster_list, labels_init_list):

  # 1: Compute Confusion matrix (for the saved features)
  cmtx = confusion_matrix2(clusters_features, labels_features, cluster_list, labels_init_list)

  # 2: Find max in each row -> cluster corresponding to each label
  map_idx = np.argmax(cmtx, axis = 0)  

  # Fill dictionary with map
  map_clu2lbl = {}
  map_lbl2clu = {}
  labels_init_list.sort()
  for i in range(0, len(map_idx)):
    map_lbl2clu[map_idx[i]] = labels_init_list[i]
    map_clu2lbl[labels_init_list[i]] = map_idx[i]
  
  # Mapping dictionary
  # map_clu2lbl -> cluster: label
  # map_lbl2clu -> label: cluster

  return map_clu2lbl, map_lbl2clu


'''Function to compute kmean clustering on the new dataset and the saved features'''
def k_mean_clustering(features_run, features_saved, labels_run, labels_saved, n_cluster, batch_size, verbose = True):

  # Define initial set of features
  labels_init_list = list(range(0, n_cluster))

  # labels_init_list = list([1, 9, 5, 0])
  # labels_init_list = list([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
  # n_cluster = len(labels_init_list)

  # Extract from the saved features the labels that we need
  features_saved_init = []
  labels_features_saved_init = []
  # Extract features of digits considered in labels_init_list
  for i in range(0, len(features_saved)):
      if labels_saved[i] in labels_init_list:
        features_saved_init.append(features_saved[i,:])
        labels_features_saved_init.append(labels_saved[i])
  
  # Convert list to nparray
  features = np.array(features_saved_init)
  features = features.astype('float32')
  labels_features = np.array(labels_features_saved_init)  

  # Concateno al vettore delle features iniziali le features della nuova batch da analizzare
  features = np.concatenate((features, features_run))
  labels_features = np.append(labels_features, labels_run).astype(int)

  # Repeat until clustering is correct
  max_iter = 1
  iter = 0
  while True:
    # KMean Clustering
    k_mean = create_k_mean(features, n_cluster)

    # Find pseudolabels for each new image
    # Pseudolabels are computed by looking at the confusion matrix of the saved dataset (where ground truth is known)
    clusters_features_saved = list(k_mean.labels_[0:len(labels_features_saved_init)])
    labels_features_saved_init = list(labels_features_saved_init)
    cluster_list = list(range(0,n_cluster))
    map_clu2lbl, map_lbl2clu = cluster_to_label(clusters_features_saved, labels_features_saved_init, cluster_list, labels_init_list) 
 
    iter += 1
    if len(map_clu2lbl) == n_cluster or iter > max_iter:
        # Exit the loop
        break

  clusters_features = k_mean.labels_

  # Compute pseudolabels
  pseudolabels = []
  for i in range(0, len(clusters_features)):
    pseudolabel = map_clu2lbl[clusters_features[i]]
    pseudolabels.append(pseudolabel)

  pseudolabels_run = pseudolabels[len(clusters_features) - batch_size: len(clusters_features)]

  err = 0 # Initialize error counter
  for i in range(len(labels_run)):
    if pseudolabels_run[i] != labels_run[i]:
      err += 1
  
  # Evaluation metrics
  if verbose:
    ComputeClusteringMetrics(features, pseudolabels, k_mean)
    ComputeEvalMetrics(labels_run, pseudolabels_run, labels_init_list)

  return pseudolabels_run, err


In [30]:
def update_ll_OL(model, features, pseudolabel):
    learn_rate = model.l_rate
    
    CheckLabelKnown(model, pseudolabel)
    
    y_true_soft = DigitToSoftmax(pseudolabel, model.label)
               
    # Prediction
    y_pred = model.predict(features)
       
    # Backpropagation
    cost = y_pred-y_true_soft
        
    for j in range(0,model.W.shape[0]):
         # Update weights
        dW = np.multiply(cost, features[j]*learn_rate)
        model.W[j,:] = model.W[j,:]-dW

    # Update biases
    db      = np.multiply(cost, learn_rate)
    model.b = model.b-db

    prediction = np.argmax(y_pred)
    return prediction



def update_ll_CWR(model, features, pseudolabel):
    learn_rate = model.l_rate
    model_batch_size = model.batch_size

    found_digit = np.zeros(10)  # reset
           
    CheckLabelKnown(model, pseudolabel)
    y_true_soft = DigitToSoftmax(pseudolabel, model.label)
        
    h = model.W.shape[0]
    w = model.W.shape[1] 

    found_digit[np.argmax(y_true_soft)] += 1  # update the digit counter
            
    # PREDICTION

    y_pred_c = softmax(np.array(np.matmul(features, model.W) + model.b))      
    y_pred_t = softmax(np.array(np.matmul(features, model.W_2) + model.b_2)) 
 
    # BACKPROPAGATION
    cost = y_pred_t-y_true_soft

    # Update weights
    for j in range(0,h):
        deltaW = np.multiply(cost, features)
        dW = np.multiply(deltaW, learn_rate)
        model.W_2[j,:] = model.W_2[j,:] - dW

    # Update biases
    db = np.multiply(cost, learn_rate)
    model.b_2 = model.b_2-db
        
        
    # If beginning of batch
    if(i%model_batch_size==0 and i!=0): 
        for k in range(0, w):
            if(found_digit[k]!=0):
                tempW = np.multiply(model.W[:,k], found_digit[k])
                tempB = np.multiply(model.b[k]  , found_digit[k])
                model.W[:,k] = np.multiply(tempW+model.W_2[:,k], 1/(found_digit[k]+1))
                model.b[k]   = np.multiply(tempB+model.b_2[k],   1/(found_digit[k]+1))
                    
        model.W_2  =  np.copy(model.W) # np.zeros((model.W.shape)) 
        model.b_2  =  np.copy(model.b) # np.zeros((model.b.shape))       
        found_digit = np.zeros(10)  # reset

    prediction = np.argmax(y_pred_c)
    return prediction

In [31]:
def trainOneEpoch(model, images, labels, features_saved, labels_saved, batch_size, ll_ALGORITHM, verbose = False):

    n_cluster = 10
    n_samples = images.shape[0]

    clust_err_array = []
    model_err_array = []

    # BATCH PROCESSING OF DATA
    n_batch = int(np.ceil(n_samples / batch_size))
    images_batch = np.array_split(images, n_batch)
    labels_batch = np.array_split(labels, n_batch)

    err_clu = 0 # Clustering error (entire epoch)
    err_mod = 0 # Model error (entire epoch)
    # err_batch = np.zeros((n_batch,2))
    pseudolabels = []
    for i in range(0, n_batch):
        print("Starting batch: {}/{}".format(i+1, n_batch))
        # Features extraction
        start1 = time.time()
        features_batch = model.ML_frozen.predict(images_batch[i].reshape((batch_size,28,28,1)), verbose = False)
        end1 = time.time()

        # Kmean clustering
        start2 = time.time()
        pseudolabels_batch, err_clu_batch = k_mean_clustering(features_batch, features_saved, labels_batch[i], labels_saved, n_cluster, batch_size, verbose = False)
        end2 = time.time()
        pseudolabels.extend(pseudolabels_batch)
        # err_batch[i, :] = err, len(labels_batch)
        err_clu += err_clu_batch
        clust_err_array.append(err_clu_batch)

        # SALVARE L'ARRAY DI PREDICTIONS
        err_mod_batch = 0
        for j in range(len(pseudolabels_batch)):
            if ll_ALGORITHM == 'OL':
                prediction = update_ll_OL(model, features_batch[j,:], pseudolabels_batch[j])
            if ll_ALGORITHM == 'CWR':
                prediction = update_ll_CWR(model, features_batch[j,:], pseudolabels_batch[j])

            if(prediction != labels_batch[i][j]):  
                err_mod_batch += 1
                # print("pred", prediction, "lab", labels_batch[i][j])

        model_err_array.append(err_mod_batch)
        err_mod += err_mod_batch

        if verbose:
            print("Features extraction took {:.3f} seconds and Kmean clustering took {:.3f} seconds, with {:.1%} accuracy ({} errors)".format(end1-start1, end2-start2, 1-err_clu_batch/batch_size, err_clu_batch))
            print("Batch Model errors {} ({:.1%} accuracy)".format(err_mod_batch, 1-err_mod_batch/batch_size))

    print("Total clustering error: {:.1%} ({}/{} errors, {:.1%} accuracy)".format(err_clu/n_samples, err_clu, n_samples, 1-err_clu/n_samples))
    print("Total model error: {:.1%} ({}/{} errors, {:.1%} accuracy)".format(err_mod/n_samples, err_mod, n_samples, 1-err_mod/n_samples))

    return clust_err_array, model_err_array

In [40]:
# Options

n_feat = 30 # Select number of features
batch_size = 5 # Clustering batch size

MODEL_PATH = 'Models/{}/'.format(n_feat)
features_saved = np.loadtxt(MODEL_PATH + 'll_features.txt')
labels_features_saved = np.loadtxt(MODEL_PATH + 'll_labels_features.txt').astype(int)
model = keras.models.load_model(MODEL_PATH + 'original_mnist_cnn.h5') # Original model 

Model_OL = Custom_Layer(model)
Model_OL.title      = 'OL'
Model_OL.filename   = 'OL'
Model_OL.l_rate     = 0.01
Model_OL.batch_size = 8

digits = np.concatenate((digits_train, digits_test))   
labels = np.concatenate((label_digits_train, label_digits_test))  

ALGORITHM = 'OL'
# ALGORITHM = 'CWR'

start3 = time.time()
clust_err_array, model_err_array = trainOneEpoch(Model_OL, digits, labels, features_saved, labels_features_saved, batch_size, ALGORITHM)
end3 = time.time()

print("Elaboration time {:.3f}".format(end3-start3))

print(clust_err_array)
print(model_err_array)

Starting batch: 1/80
Starting batch: 2/80
New digit detected -> 8
New digit detected -> 9
Starting batch: 3/80
New digit detected -> 7
Starting batch: 4/80
Starting batch: 5/80
New digit detected -> 6
Starting batch: 6/80
Starting batch: 7/80
Starting batch: 8/80
Starting batch: 9/80
Starting batch: 10/80
Starting batch: 11/80
Starting batch: 12/80
Starting batch: 13/80
Starting batch: 14/80
Starting batch: 15/80
Starting batch: 16/80
Starting batch: 17/80
Starting batch: 18/80
Starting batch: 19/80
Starting batch: 20/80
Starting batch: 21/80
Starting batch: 22/80
Starting batch: 23/80
Starting batch: 24/80
Starting batch: 25/80
Starting batch: 26/80
Starting batch: 27/80
Starting batch: 28/80
Starting batch: 29/80
Starting batch: 30/80
Starting batch: 31/80
Starting batch: 32/80
Starting batch: 33/80
Starting batch: 34/80
Starting batch: 35/80
Starting batch: 36/80
Starting batch: 37/80
Starting batch: 38/80
Starting batch: 39/80
Starting batch: 40/80
Starting batch: 41/80
Starting ba

[20, 25, 10, 18]
[37, 51, 44, 54]


In [34]:
#import importlib
#importlib.reload(Kmeans_lib)

#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"  







feat = features_saved[1,:]

##(features_saved.all)
#max(features_saved)

feat.dtype

np.amin(feat)
np.amax(feat)

feat.shape
feat

X = digits_train[1,:]
YY = Model_OL.ML_frozen.predict(X.reshape((1,28,28,1)), verbose = False)

# YY2 = np.array(YY as features_saved)

np.amin(YY)
np.amax(YY)

#from simulation_lib import *


# IDEE METRICHE PER SIMULAZIONE 

#plot_barChart(Model_OL)
#plot_confMatrix(Model_OL)
#plot_table(Model_OL)

# from lib import simulation_lib as sim_lib
# sim_lib.plot_barChart(Model_OL)
# sim_lib.plot_confMatrix(Model_OL)
# sim_lib.plot_table(Model_OL)

4.1939554

In [35]:
# OLD
def trainOneEpoch_CWR(model, images, labels, features_saved, labels_saved, batch_size, found_digit):
                 
    n_cluster = 10
    n_samples = images.shape[0]  #test_samples
     
    prediction_vec = np.zeros(test_samples) # non so se serve in CWR update


    # BATCH PROCESSING OF DATA
    n_batch = int(np.ceil(n_samples / batch_size))
    images_batch = np.array_split(images, n_batch)
    labels_batch = np.array_split(labels, n_batch)

    err_clu = 0                    
    err_mod = 0
    err_batch = np.zeros((n_batch,2))
    pseudo_labels = []

    for i in range(0, n_batch):
        print("Starting batch: {}/{}".format(i+1, n_batch))
        # Features extraction
        start1 = time.time()
        features_batch = model.ML_frozen.predict(images_batch[i].reshape((batch_size,28,28,1)), verbose = False)
        end1 = time.time()

        # Kmean clustering
        start2 = time.time()
        pseudo_labels_batch, err = k_mean_clustering(features_batch, features_saved, labels_batch[i], labels_saved, n_cluster, batch_size)
        end2 = time.time()
        pseudo_labels.extend(pseudo_labels_batch)
        err_batch[i, :] = err, len(labels_batch)
        err_clu += err

        ## Fino qui sopra CWR e OL sono praticamente uguali e quindi si potrebbero unire

        

        print("Features extraction took {:.3f} seconds and Kmean clustering took {:.3f} seconds, with {:.1%} accuracy ({} errors)".format(end1-start1, end2-start2, 1-err/batch_size, err))

    # ONLINE-LEARNING
    print('**********************************\nPerforming training CWR \n ')  
                   
    # Cycle over all input samples
    #for i in range(0, test_samples):

 
        
    # Metrics

    print('\n*******************************************************************************')
    print('***** Model batch accuracies: \n')
    
    ComputeEvalMetrics(y_test, prediction_vec, list(range(0, n_cluster)))

    
    return np.array(pseudo_labels).astype(int), prediction_vec, err_clu, err_mod